In [1]:
import pandas as pd
import numpy as np
import os
from elo import add_elos

In [2]:
# Read data from Excel files and join vertically
path = os.path.join(os.getcwd(), "match data")
cols = ['Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts','Comment','B365W','B365L','PSW','PSL']
df_list = [pd.read_excel(os.path.join(path, file)).reindex(columns=cols) for file in os.listdir(path)]
df = pd.concat(df_list, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
# Save working copy of df 
df_copy = df.copy()

In [ ]:
# Uncomment and run to reset df back to original state
# df = df_copy

In [ ]:
# Sort df by date
df = df.sort_values(by='Date')

In [ ]:
# Remove whitespace from df entries
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

C:\Users\olive\AppData\Local\Temp\ipykernel_20048\1583458003.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# Convert rank column values to floats
df['WRank'] = pd.to_numeric(df['WRank'], errors='coerce')
df['LRank'] = pd.to_numeric(df['LRank'], errors='coerce')

In [ ]:
# Drop all matches that lack data, weren't completed or were played on carpet
df = df.dropna()
df = df[df['Comment'] == 'Completed']
df = df[df['Surface'] != 'Carpet']
df = df.reset_index(drop=True).drop(columns=['Comment'])

In [ ]:
# Convert winner/loser columns to neutral player columns (ordered alphabetically)
player_A = df[['Winner','Loser']].min(axis=1)
player_B = df[['Winner','Loser']].max(axis=1)

In [ ]:
# Create target variable
A_won = df['Winner'] == player_A

In [ ]:
# Update dataframe to reflect change to player columns
# Add column for target variable in the process
df[['Player A','Player B','A Rank','B Rank','A Pts','B Pts','A Bet365 Odds',
    'B Bet365 Odds','A Pinnacle Odds','B Pinnacle Odds','A Won']] = pd.DataFrame({
        'Player A': player_A,
        'Player B': player_B,
        'A Rank': df['WRank'].where(A_won, df['LRank']),
        'B Rank': df['LRank'].where(A_won, df['WRank']),
        'A Pts': df['WPts'].where(A_won, df['LPts']),
        'B Pts': df['LPts'].where(A_won, df['WPts']),
        'A Bet365 Odds': df['B365W'].where(A_won, df['B365L']),
        'B Bet365 Odds': df['B365L'].where(A_won, df['B365W']),
        'A Pinnacle Odds': df['PSW'].where(A_won, df['PSL']),
        'B Pinnacle Odds': df['PSL'].where(A_won, df['PSW']),
        'A Won': A_won.astype(int)
    })
df = df.drop(columns=['Winner','Loser','WRank','LRank','WPts','LPts','B365W','B365L','PSW','PSL'])

In [ ]:
# Change df to 'long' format where each row is a match from 1 player's perspective
df_long_A = df.rename(columns={"Player A": "Player", "Player B": "Opponent"})
df_long_A["Player Won"] = df_long_A["A Won"]
df_long_B = df.rename(columns={"Player B": "Player", "Player A": "Opponent"})
df_long_B["Player Won"] = 1 - df_long_B["A Won"]
df_long = pd.concat([df_long_A, df_long_B], ignore_index=True)

In [ ]:
# Sort long df by date
df_long = df_long.sort_values(["Date"]).reset_index(drop=True)

In [ ]:
# Add H2H winrate feature
df_long["H2H Wins"] = (
    df_long.groupby(["Player", "Opponent"])["Player Won"]
           .transform(lambda x: x.shift().cumsum().fillna(0))
)
df_long['H2H Matches'] = df_long.groupby(['Player', 'Opponent'])['Player Won'].cumcount()
df_long['H2H Winrate'] = df_long['H2H Wins'] / df_long['H2H Matches'].clip(lower=1)

In [ ]:
# Add 5 match winrate feature
df_long["5 Match Winrate"] = (
    df_long.groupby("Player")["Player Won"]
           .transform(lambda x: x.shift().rolling(5, min_periods=5).mean())
)

In [ ]:
# Add 10 match winrate feature
df_long["10 Match Winrate"] = (
    df_long.groupby("Player")["Player Won"]
           .transform(lambda x: x.shift().rolling(10, min_periods=10).mean())
)

In [ ]:
# Add 20 match winrate feature
df_long["20 Match Winrate"] = (
    df_long.groupby("Player")["Player Won"]
           .transform(lambda x: x.shift().rolling(20, min_periods=20).mean())
)

In [ ]:
# Add surface-specific winrate feature
df_long["Surface Wins"] = (
    df_long.groupby(["Player","Surface"])["Player Won"]
           .transform(lambda x: x.shift().cumsum().fillna(0))
)
df_long["Surface Matches"] = df_long.groupby(["Player","Surface"]).cumcount()
df_long["Surface Winrate"] = df_long["Surface Wins"] / df_long["Surface Matches"].clip(lower=1)

In [ ]:
# Add career winrate feature
df_long["Career Wins"] = (
    df_long.groupby("Player")["Player Won"]
           .transform(lambda x: x.shift().cumsum().fillna(0))
)
df_long["Career Matches"] = df_long.groupby("Player").cumcount()
df_long["Career Winrate"] = df_long["Career Wins"] / df_long["Career Matches"].clip(lower=1)

In [ ]:
# Add seasonal winrate feature
df_long["Year"] = df_long["Date"].dt.year
df_long["Season Wins"] = (
    df_long.groupby(["Player","Year"])["Player Won"]
           .transform(lambda x: x.shift().cumsum().fillna(0))
)
df_long["Season Matches"] = df_long.groupby(["Player","Year"]).cumcount()
df_long["Season Winrate"] = df_long["Season Wins"] / df_long["Season Matches"].clip(lower=1)

In [ ]:
# Add days since last match feature
df_long["Days since last match"] = (
    df_long.groupby("Player")["Date"].diff().dt.days
)

In [ ]:
df_long_A = df_long.rename(columns={
    "Player": "Player A",
    "Opponent": "Player B",
    "H2H Winrate": "A H2H Winrate",
    "5 Match Winrate": "A 5 Match Winrate",
    "10 Match Winrate": "A 10 Match Winrate",
    "20 Match Winrate": "A 20 Match Winrate",
    "Surface Winrate": "A Surface Winrate",
    "Career Winrate": "A Career Winrate",
    "Season Winrate": "A Season Winrate",
    "Surface Matches": "A Surface Matches",
    "Career Matches": "A Career Matches",
    "Season Matches": "A Season Matches",
    "Days since last match": "A Days since last match"
})

In [ ]:
# Create long df for player B
df_long_B = df_long.rename(columns={
    "Player": "Player B",
    "Opponent": "Player A",
    "H2H Winrate": "B H2H Winrate",
    "5 Match Winrate": "B 5 Match Winrate",
    "10 Match Winrate": "B 10 Match Winrate",
    "20 Match Winrate": "B 20 Match Winrate",
    "Surface Winrate": "B Surface Winrate",
    "Career Winrate": "B Career Winrate",
    "Season Winrate": "B Season Winrate",
    "Surface Matches": "B Surface Matches",
    "Career Matches": "B Career Matches",
    "Season Matches": "B Season Matches",
    "Days since last match": "B Days since last match"
})

In [ ]:
# Merge new features into original df
# H2H matches is added only once as it is the same for both players
df = (
    df.merge(df_long_A[["Date","Player A","Player B","A H2H Winrate",
                        "A 5 Match Winrate","A 10 Match Winrate",
                        "A 20 Match Winrate","A Surface Winrate",
                        "A Career Winrate","A Season Winrate",
                        "A Surface Matches","A Career Matches",
                        "A Season Matches","A Days since last match"]],
             on=["Date","Player A","Player B"], how="left")
      .merge(df_long_B[["Date","Player A","Player B","B H2H Winrate",
                        "B 5 Match Winrate","B 10 Match Winrate",
                        "B 20 Match Winrate","B Surface Winrate",
                        "B Career Winrate","B Season Winrate",
                        "B Surface Matches","B Career Matches",
                        "B Season Matches","B Days since last match",
                        "H2H Matches"]],
             on=["Date","Player A","Player B"], how="left")
)

In [ ]:
# Add elo features
df = add_elos(df)

In [ ]:
# Convert date column to year and month columns
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df = df.drop(columns=['Date'])

In [ ]:
# Remove player features
df = df.drop(columns=["Player A", "Player B"])

In [ ]:
# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Series', 'Court', 'Surface', 'Round'], dtype=int)

In [ ]:
df.to_csv("df.csv", index=False)